In [1]:
import findspark
findspark.init()


In [2]:
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()



In [3]:
# read from kafka, here spark is consumer for kafka topic called invoices
# spark streaming works as dataframe/sql
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "192.168.93.128:9092")\
  .option("subscribe", "invoices")\
  .load()
 
    

In [5]:
# .show/print will not work directily due to stream..
# linesDf.show() # worn't work
kafkaDf.printSchema() # works

# key is kafka key, in binary format
# value is kafka value, in binary format
# topic string
# parition, integer
# offer long 
# timestamp - longint in ms
# timestampType - Source Time, Record write time

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# now convert kafka value which is in bytes to STRING, we ignore the key for now...
# now we pick only value from the stream..
invoiceJsonRawDf = kafkaDf.selectExpr("timestamp", "CAST(value AS STRING)")
invoiceJsonRawDf.printSchema() # we get only value as string

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: string (nullable = true)



In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, DateType
# json is object, spark DF needs schema 

schema = StructType(
        [
            StructField("InvoiceNo", IntegerType(), True),
            StructField("StockCode", StringType(), True),
            StructField("Quantity", IntegerType(), True),
            StructField("Description", StringType(), True),
            StructField("InvoiceDate", StringType(), True),
            #StructField("InvoiceDate", DateType(), True),
            StructField("UnitPrice", DoubleType(), True),
            StructField("CustomerID", IntegerType(), True),
            StructField("Country", StringType(), True),
        ]
)

#{"InvoiceNo": 495774, "StockCode": "84406G", "Quantity": 2, "Description": "TODO", "InvoiceDate": "05/22/2021 00:36", "UnitPrice": 2.0, "CustomerID": 17850, "Country": "AT"}

# replacing json string with a json object with schema
# now value is a column, it contains a struct
jsonDf = invoiceJsonRawDf.withColumn("value", F.from_json("value", schema))

# now we will extract value which struct type ewith all schema field mention, to specific columns
#InvoiceNo, StockCode, ....
invoiceDf = jsonDf.select("timestamp", F.col("value.*")) 

In [ ]:
# run cell by cell, than all...
# dataframe specific, raw 
echoOnconsole = invoiceDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [ ]:
filteredDf = invoiceDf.filter("Quantity >= 6") 

echoOnconsole = filteredDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [ ]:
# groupby for count, unique items, not count of Quantity 
# 2 apples, 3 orangles = answer 2
groupByItemCount = invoiceDf.groupBy("InvoiceNo").count()

echoOnconsole = groupByItemCount\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [8]:
# Give me how much money generated selling goods for last 60 seconds/1 minutes
invoiceDf = invoiceDf.withColumn("Amount", F.col("Quantity") * F.col("UnitPrice") )
# below code is not right solution, group by, by last last 6o seconds
# groupByItemCount = invoiceDf.groupBy("InvoiceNo")....

windowedAmountSum = invoiceDf.groupBy(F.window(invoiceDf.timestamp, 
                                              "60 seconds", 
                                               "60 seconds"), invoiceDf.Amount).sum()

echoOnconsole = windowedAmountSum\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [8]:
# Give me how much money generated selling goods for last 60 seconds/1 minutes
invoiceDf = invoiceDf.withColumn("Amount", F.col("Quantity") * F.col("UnitPrice") )
# below code is not right solution, group by, by last last 6o seconds
# groupByItemCount = invoiceDf.groupBy("InvoiceNo")....

windowedAmountSum = invoiceDf.groupBy(F.window(invoiceDf.timestamp, 
                                              "60 seconds", 
                                               "60 seconds"))\
                              .agg(F.sum("Amount"))

echoOnconsole = windowedAmountSum\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

In [ ]:
echoOnconsole.awaitTermination()

# later you can terminal the jupyter